# object detction using yolo8

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.2/716.2 kB 13.3 MB/s eta 0:00:00


In [24]:
#final


import torch
import torchvision.transforms as transforms
import cv2
from google.colab.patches import cv2_imshow

# Load pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Access the video file
video_path = '/content/drive/My Drive/DL_HW4/Street_vd.MOV'
cap = cv2.VideoCapture(video_path)

# Define the output video path
output_video_path = '/content/drive/My Drive/DL_HW4/Street_vd_annotated.mp4'

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process video frames
with torch.no_grad():
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Perform inference
        results = model(frame)

        # Filter detections for vehicles
        vehicle_detections = results.pred[0][results.pred[0][:, 5] == 2]  # Class ID 2 corresponds to 'car'

        # Draw bounding boxes and labels on frame for vehicle detections
        for detection in vehicle_detections:
            bbox = detection[:4].cpu().numpy().astype(int)
            class_id = int(detection[5])
            confidence = float(detection[4])
            class_name = model.names[class_id]

            # Unpack bounding box coordinates
            x1, y1, x2, y2 = bbox

            # Draw bounding box and label if confidence threshold is met
            if confidence > 0.5:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{class_name} {confidence:.2f}', (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Write the annotated frame to the output video
        out.write(frame)

        # Display the annotated frame
        cv2_imshow(frame)

        if cv2.waitKey(1) == 27:
            break

# Release video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.